In [ ]:
import pandas as pd
from concurrent.futures import ProcessPoolExecutor
from difflib import SequenceMatcher
import re
from collections import defaultdict
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import psycopg2
import nltk
from nltk.tokenize import PunktSentenceTokenizer, word_tokenize
import spacy
from tqdm import tqdm
import stanza

In [ ]:
# Initialize Stanza pipeline for English with only tokenization (no parsing to keep it efficient)
stanza.download('en')
nlp_stanza = stanza.Pipeline(lang='en', processors='tokenize', tokenize_pretokenized=False)
nlp_spacy = spacy.load("en_core_web_sm")

# Function to tokenize sentences using Stanza with fallback to SpaCy
def tokenize_with_fallback(text, max_sentence_length=300):
    # First attempt tokenization with Stanza
    sentences = stanza_tokenize(text)
    
    # Check if any sentence exceeds the max length and re-tokenize with SpaCy if needed
    final_sentences = []
    for sentence in sentences:
        if len(sentence) > max_sentence_length:
            spacy_sentences = spacy_tokenize(sentence)
            final_sentences.extend(spacy_sentences)
        else:
            final_sentences.append(sentence)
    
    return final_sentences

# Function to tokenize using Stanza
def stanza_tokenize(text):
    doc = nlp_stanza(text)
    return [sentence.text for sentence in doc.sentences]

# Function to tokenize using SpaCy
def spacy_tokenize(text):
    doc = nlp_spacy(text)
    return [sent.text for sent in doc.sents]

In [ ]:
# Define the lists of keywords
keywords_list1 = {
    'alzheimer', 'dementia', 'demented', 'mild cognitive impairment', 'amnesia', 'amnestic', 'frontotemporal', 'neurocognit',
    'lewy body', 'aphasia', 'memory', 'Donepezil', 'Aricept', 'Galantamine', 'Reminyl', 'Razadyne', 'Rivastigmine', 'Exelon', 
    'Memantine', 'Namenda', 'Namzaric', 'Aducanumab', 'Aduhelm', 'Lecanemab', 'Leqembi', 'confus', 'amnesia', 'forget', 
    'forgot', 'word', 'disorientation', 'attention', 'problem solving', 'executive dysfunction', 
    'poor safety awareness', 'mood', 'speech', 'neuropsych', 'visuospatial', 'Mini-Cog', 'Mini-Mental State Examination', 
    'Montreal cognitive assessment', 'clinical dementia rating', 'Saint Louis University Mental Status Exam', 'cogniti', 
    'memory', 'alert', 'oriented', 'recall', 'attention', 'processing speed', 'verbal fluency', 'encoding', 'visuospatial', 
    'naming', 'orientation'
}

keywords_list2 = {'MCI', 'FTD', 'LBD', 'MMSE', 'GPCOG', 'AD8', 'SLUMS', 'MoCA', 'BDS', 'CDR', 'BNT'}

# Convert keywords to lowercase for easier searching (except for keywords_list2 as we need exact matching)
keywords_list1 = {kw.lower() for kw in keywords_list1}


In [ ]:
# Establish a connection to the database
conn = psycopg2.connect(
)

# SQL query to retrieve the relevant notes data
# exclude radiology reports
query = """

SELECT empi, report_date, notetxt
FROM adrd.adrd_study_700_notes_all
WHERE notetype!= 'rad' ;

"""

# Load the data into a pandas DataFrame
notes_data = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

In [ ]:
# Clean the notes by removing special characters and spaces
notes_data['notetxt'] = notes_data['notetxt'].str.replace(r'[\r\n]+', ' ', regex=True)
notes_data['notetxt'] = notes_data['notetxt'].str.replace(r'\s+', ' ', regex=True).str.strip()
notes_data['notetxt'] = notes_data['notetxt'].apply(lambda text: re.sub(r'-', '', text))
notes_data['notetxt'] = notes_data['notetxt'].apply(lambda text: re.sub(r'\(.*?\)', '', text))
notes_data['notetxt'] = notes_data['notetxt'].apply(lambda text: re.sub(r'\*', '', text))
notes_data['notetxt'] = notes_data['notetxt'].apply(lambda text: re.sub(r'(.)\1{2,}', r'\1\1', text))


In [ ]:
notes_data.info()

In [ ]:
# Function to search for keywords in a sentence (for keywords_list1, we use partial matching)
def search_keywords_list1(sentence, keywords):
    sentence_lower = sentence.lower()
    return any(kw in sentence_lower for kw in keywords)

# Function to search for keywords in a tokenized sentence (for keywords_list2, we use exact matching)
def search_keywords_list2(tokenized_sentence, keywords):
    return any(word in keywords for word in tokenized_sentence)

# Function to get unique context indices
def get_context_indices(sentences, idx):
    context_indices = {idx}  # Include the sentence itself
    if idx > 0:  # Add previous sentence index if exists
        context_indices.add(idx - 1)
    if idx < len(sentences) - 1:  # Add next sentence index if exists
        context_indices.add(idx + 1)
    return context_indices

# Process the notes and extract relevant sentences with unique context indices
def extract_relevant_sentences(note, keywords_list1, keywords_list2, seen_sentences):
    sentences = tokenize_with_fallback(note, max_sentence_length)  # Using Stanza tokenizer
    unique_indices = set()

    # Identify relevant sentences and their context indices
    for idx, sentence in enumerate(sentences):
        tokenized_sentence = sentence.split()

        if (search_keywords_list1(sentence, keywords_list1) or search_keywords_list2(tokenized_sentence, keywords_list2)) and sentence not in seen_sentences:
            seen_sentences.add(sentence)
            unique_indices.update(get_context_indices(sentences, idx))

    # Retrieve sentences based on unique indices and maintain order
    relevant_sentences = [sentences[i] for i in sorted(unique_indices)]
    
    # Join the relevant sentences into a single string
    return " ".join(relevant_sentences)

# Sequential processing function with progress tracking
def process_notes_data(notes_data):
    # Initialize empty list for filtered texts and counter
    filtered_texts = []
    print_interval = 5000  # Set the interval for printing progress

    for i, note in enumerate(notes_data['notetxt']):
        empi - notes_data.loc[i, 'empi']

         # Initialize a seen_sentences set for each patient
        if empi not in seen_sentences:
            seen_sentences[empi] = set()       
            
        # Extract relevant sentences for each note
        filtered_texts.append(extract_relevant_sentences(note, keywords_list1, keywords_list2, seen_sentences[empi]))
        
        # Print progress every 5000 notes
        if (i + 1) % print_interval == 0:
            print(f"Processed {i + 1} notes out of {len(notes_data['notetxt'])}")

    notes_data['filtered_text'] = filtered_texts

    # Group by patient and report date and concatenate notes
    grouped_notes = notes_data.groupby(['empi', 'report_date'])['filtered_text'].apply(lambda x: ' '.join(x)).reset_index()
    return grouped_notes

In [ ]:
notes_data.info()

In [ ]:
# Initialize the seen_sentences dictionary to track identified sentences per patient
seen_sentences = {}

grouped_notes = process_notes_data(notes_data)

# Optionally, save the result to a CSV file
grouped_notes.to_csv('data/adrd_filtered_patient_notes_stanza.csv', index=False)


In [ ]:
# Load filtered notes data
filtered_notes = pd.read_csv("data/adrd_filtered_patient_notes_stanza_v2.csv")

# Filter out rows with no actual clinical notes
filtered_notes = filtered_notes[
    (filtered_notes['filtered_text'].notna()) &
    (filtered_notes['filtered_text'].str.strip() != '') &
    (filtered_notes['filtered_text'].str.lower() != 'null')
]

filtered_notes['filtered_text'] = filtered_notes['filtered_text'].replace('Null', '')

# Load other tables assuming they are saved as CSV files from the SQL queries
diagnosis = pd.read_csv("data/adrd_combined_diagnosis.csv")  # Result of Diagnosis query
medications = pd.read_csv("data/adrd_combined_medications.csv")  # Result of Medications query
problem_list = pd.read_csv("data/adrd_combined_problems.csv")  # Result of Problem List query
dob_data = pd.read_csv("data/adrd_dob.csv")  # Contains EMPI and date of birth


# Rename columns for consistency if necessary
filtered_notes.rename(columns={'report_date': 'record_date'}, inplace=True)
diagnosis.rename(columns={'diagnosis_date': 'record_date'}, inplace=True)
medications.rename(columns={'medication_date': 'record_date'}, inplace=True)
problem_list.rename(columns={'diagnosis_date': 'record_date'}, inplace=True)

# Convert dates to datetime for consistency
dob_data['dob'] = pd.to_datetime(dob_data['dob'])
filtered_notes['record_date'] = pd.to_datetime(filtered_notes['record_date'])
diagnosis['record_date'] = pd.to_datetime(diagnosis['record_date'])
medications['record_date'] = pd.to_datetime(medications['record_date'])
problem_list['record_date'] = pd.to_datetime(problem_list['record_date'])


# Perform full outer join to include all patients and dates from any source
merged_data = pd.merge(merged_data, diagnosis[['empi', 'record_date', 'combined_diagnosis']],
                       on=['empi', 'record_date'], how='outer')
merged_data = pd.merge(merged_data, medications[['empi', 'record_date', 'medications']],
                       on=['empi', 'record_date'], how='outer')
merged_data = pd.merge(merged_data, problem_list[['empi', 'record_date', 'combined_diagnosis']],
                       on=['empi', 'record_date'], how='outer', suffixes=('', '_problem_list'))
merged_data = pd.merge(filtered_notes, dob_data, on='empi', how='outer')
# Fill missing filtered notes with explicit 'Null' for clarity
merged_data['filtered_text'].fillna('Null', inplace=True)

# Calculate age only where dob is available and record_date is not missing
merged_data['age'] = merged_data.apply(lambda x: (x['record_date'] - x['dob']).days // 365 if pd.notnull(x['dob']) and pd.notnull(x['record_date']) else None, axis=1)

merged_data.to_csv("data/final_patient_data_table_intermediate.csv", index=False)



In [ ]:

# Rename columns to match the output table requirements
merged_data.rename(columns={
    'filtered_text': 'clinical_notes',
    'combined_diagnosis': 'diagnoses',
    'combined_diagnosis_problem_list': 'problem_lists'
}, inplace=True)

def aggregate_info(row):
    parts = []

    age = int(row['age']) if pd.notna(row['age']) else "unknown"
    
    parts.append(f"The patient was {age} years old. On {row['record_date']}, the following were documented in the EHRs: ")
    
    if pd.notna(row['diagnoses']):
        parts.append(f"The patient was diagnosed with: {row['diagnoses']}")
        
    if pd.notna(row['medications']):
        parts.append(f"The patient received the following medications: {row['medications']}")
        
    if pd.notna(row['problem_lists']):
        parts.append(f"The problem list included: {row['problem_lists']}")

    if pd.notna(row['clinical_notes']) and row['clinical_notes'] != "Null":
        parts.append(f"The clinical notes documented: {row['clinical_notes']}")
    
    return "\n".join(parts)

merged_data['record_date'] = pd.to_datetime(merged_data['record_date']).dt.date
merged_data['medical_summary'] = merged_data.apply(aggregate_info, axis=1)

# Reorder columns for the final output table
final_table = merged_data[['empi', 'record_date', 'age', 'clinical_notes', 
                           'diagnoses', 'medications', 'problem_lists', 'medical_summary']]
# Save the final table to a new CSV file
final_table.to_csv("data/final_patient_data_table.csv", index=False)

print("Final table created successfully!")